# Tu Misión: Entrenar una Red Neuronal para Reconocer Dígitos

¡Bienvenido/a al Desafío MNIST! En este notebook aprenderás a construir, entrenar y evaluar una red neuronal profunda capaz de reconocer dígitos escritos a mano. Utilizaremos Python y TensorFlow/Keras para abordar este reto clásico de Machine Learning y Deep Learning.

**Fases del proyecto:**
1. Configuración del entorno y carga de librerías
2. Carga y exploración del dataset MNIST
3. Visualización de imágenes del dataset
4. Preprocesamiento de los datos
5. Construcción de la arquitectura del modelo
6. Visualización del resumen del modelo
7. Compilación y entrenamiento del modelo
8. Evaluación del modelo en el conjunto de prueba
9. Visualización de gráficos de precisión y pérdida
10. Análisis de sobreajuste y preguntas de reflexión

Cada sección incluye explicaciones y preguntas de análisis para profundizar tu comprensión.

In [ ]:
# 1. Configuración del Entorno y Carga de Librerías
# Importamos las librerías necesarias para el proyecto.
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

In [ ]:
# 2. Carga y Exploración del Dataset MNIST
# Cargamos el dataset MNIST usando Keras y verificamos las dimensiones de los datos.
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(f"Imágenes de entrenamiento: {x_train.shape}")
print(f"Imágenes de prueba: {x_test.shape}")
print(f"Etiquetas de entrenamiento: {y_train.shape}")
print(f"Etiquetas de prueba: {y_test.shape}")

In [ ]:
# 3. Visualización de Imágenes del Dataset
# Mostramos 5 imágenes diferentes del conjunto de entrenamiento junto con sus etiquetas.
fig, axes = plt.subplots(1, 5, figsize=(12, 3))
for i in range(5):
    axes[i].imshow(x_train[i], cmap='gray')
    axes[i].set_title(f"Etiqueta: {y_train[i]}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

### Pregunta de Análisis 1
¿Encuentras variaciones en la escritura de un mismo número (por ejemplo, el "7" o el "2")? ¿Por qué crees que estas variaciones representan un desafío para un algoritmo?

*Respuesta:* 
Sí, se observan variaciones significativas en la forma en que diferentes personas escriben el mismo número. Estas diferencias pueden incluir grosor, inclinación, tamaño y estilo. Para un algoritmo, estas variaciones dificultan la tarea de identificar patrones consistentes, ya que debe aprender a reconocer la esencia del número independientemente de las diferencias individuales en la escritura. Esto hace que el problema sea complejo y requiera modelos robustos capaces de generalizar.

In [ ]:
# 4. Preprocesamiento de los Datos
# Normalizamos los valores de los píxeles, aplanamos las imágenes y realizamos one-hot encoding de las etiquetas.
x_train_norm = x_train.astype('float32') / 255.0
x_test_norm = x_test.astype('float32') / 255.0

x_train_flat = x_train_norm.reshape(-1, 28*28)
x_test_flat = x_test_norm.reshape(-1, 28*28)

y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

print(f"Datos de entrenamiento normalizados y aplanados: {x_train_flat.shape}")
print(f"Etiquetas codificadas (one-hot): {y_train_cat.shape}")

In [ ]:
# 5. Construcción de la Arquitectura del Modelo
# Definimos la red neuronal usando la API Sequential de Keras.
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# 6. Visualización del Resumen del Modelo
# Mostramos la arquitectura y el número de parámetros del modelo.
model.summary()

In [ ]:
# 7. Compilación y Entrenamiento del Modelo
# Compilamos el modelo y lo entrenamos usando los datos preprocesados.
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    x_train_flat, y_train_cat,
    epochs=20,
    batch_size=128,
    validation_split=0.1,
    verbose=2
)

In [ ]:
# 8. Evaluación del Modelo en el Conjunto de Prueba
# Evaluamos el modelo usando los datos de prueba y reportamos la precisión final.
test_loss, test_acc = model.evaluate(x_test_flat, y_test_cat, verbose=2)
print(f"Precisión final en el conjunto de prueba: {test_acc:.4f}")

In [ ]:
# 9. Visualización de Gráficos de Precisión y Pérdida
# Graficamos la precisión y la pérdida de entrenamiento y validación a lo largo de las épocas.
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión del Modelo')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida del Modelo')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.tight_layout()
plt.show()

### Pregunta de Análisis 2
¿La precisión de entrenamiento sigue mejorando mientras que la de validación se estanca o empeora? ¿La pérdida de entrenamiento disminuye mientras que la de validación aumenta? Si respondiste sí, estás viendo un claro signo de sobreajuste.

*Respuesta:* 
Si los gráficos muestran que la precisión de entrenamiento sigue aumentando pero la de validación se mantiene o disminuye, y la pérdida de validación aumenta mientras la de entrenamiento baja, esto indica sobreajuste. El modelo está "memorizando" los datos de entrenamiento y pierde capacidad de generalización sobre datos nuevos.

### Pregunta de Análisis 3
Explica con tus propias palabras qué es el sobreajuste. ¿Por qué es un problema que el modelo funcione perfectamente con los datos de entrenamiento pero no tan bien con los de validación?

*Respuesta:* 
El sobreajuste ocurre cuando un modelo aprende demasiado bien los detalles y el ruido de los datos de entrenamiento, perdiendo la capacidad de generalizar a datos nuevos. Es un problema porque el objetivo de un modelo de Machine Learning es funcionar bien con datos que nunca ha visto. Si el modelo solo es preciso con los datos de entrenamiento, su utilidad en el mundo real es limitada.

**Estrategias para reducir el sobreajuste:**
- **Dropout:** Consiste en desactivar aleatoriamente algunas neuronas durante el entrenamiento, lo que obliga al modelo a no depender de rutas específicas y mejora la generalización.
- **Regularización L2:** Penaliza los pesos grandes en la función de pérdida, ayudando a que el modelo sea más simple y menos propenso a memorizar el ruido.
- **Aumentar el dataset:** Más datos permiten que el modelo aprenda patrones más generales y robustos, reduciendo el riesgo de sobreajuste.

Estas estrategias funcionan porque ayudan al modelo a enfocarse en patrones generales y no en detalles específicos de los datos de entrenamiento.